#### **Welcome to Assignment 1 on Deep Learning for Computer Vision.**
This assignment is based on the content you learned in Week-4.

#### **Instructions**
1. Use Python 3.x to run this notebook
2. Write your code only in between the lines 'YOUR CODE STARTS HERE' and 'YOUR CODE ENDS HERE'. You should not change anything else in the code cells, if you do, the answers you are supposed to get at the end of this assignment might be wrong.
3. Read documentation of each function carefully.
4. All the Best!

We will be working with Pytorch operations

You will perform basic elementwise operations between two matrices

Step1: Rearrange the give input x into a tensor of shape (10,5,5) and store it in x_resize \\
Step2: Generate a gaussian random tensor (mean =0, variance=1) of shape (5,5) and store it in variable y1 \\
Step3:  Generate a uniform random tensor (interval [0,1]) of shape (5,5) and store it in variable y2 \\
Step4: Perform elementwise multiplication between x_resize and y1,such that the output will be of shape (10,5,5) and store it in variable mul_output1 \\
Step5: Sum over the last two dimensions of mul_output1 and store it in final_output1. The shape of final_output1 should be ([10]) \\
Step6: Repeat Step4 and Step5 for x_resize and y2

In [ ]:
import torch
#DO NOT CHANGE THE SEED
torch.manual_seed(2)

x = torch.randint(low=0,high=256,size=(1,250))
##Your code here##

x_resize = x.view(10, 5, 5)

y1 = torch.randn(5, 5)

y2 = torch.rand(5, 5)

mul_output1 = x_resize * y1.unsqueeze(0)

final_output1 = mul_output1.sum(dim=(1, 2))

mul_output2 = x_resize * y2.unsqueeze(0)

final_output2 = mul_output2.sum(dim=(1, 2))

## Your code ends here##

print(torch.mean(final_output1+final_output2))

tensor(2052.6074)


### Q1. What is mean of the sum of final_output1 and final_output2? (Choose the closest value)
a) 1000 \\
b) 2000 \\
c) 3000 \\
d) 4000

---

You will calculate L2 loss between predicted values and labels \\


Step1: Resize the final_output1 and final_output2 from the above code to shape (10,1) and store it in o1 and o2 respectively \\
Step2: Concatenate these two vectors o1 and o2 such that the shape of the resultant tensor is (10,2) and store it in final_output \\
Step3: Pass the tensor final_output as input to ReLU function and store it in logits variable \\
Step4: Calculate Eucledian norm of the difference (labels-logits)  across dimension = 1 and store it in eucledian_dist. Your output shape should be ([10])


In [ ]:
##DO NOT CHANGE SEED##
import torch.nn.functional as F
torch.manual_seed(0)

labels = torch.randint(low=0,high=2000,size=(10,2)).type(torch.float)

## Your code starts here##
o1 = final_output1.view(10, 1)
o2 = final_output2.view(10, 1)
final_output = torch.cat((o1, o2), dim=1)

logits = F.relu(final_output)

eucledian_dist = torch.norm(labels - logits, dim=1)

## Your code ends here
print(eucledian_dist.shape)
loss = torch.sum(eucledian_dist)
print(loss)

torch.Size([10])
tensor(12358.2744)


### Q2. What is sum of the elements of the eucledian norm? (Choose the closest value)
a) 12000 \\
b) 13000 \\
c) 10000000 \\
d) 20000000

---

You will calculate gradient of a vector w.r.t to a function \\


Step1: Generate gaussian random tensor (mean =0, variance=1) of shape(3) and store it in w \\
Step2: Generate a uniform random tensor (interval [0,1]) of shape(3,3) and store it in X \\
Step3: Calculate the matrix vector product between X and w and store it in pred \\
Step4: Calculate L1 norm of the difference (labels - pred) and store it in temp. The output should be of shape [3] \\
Step5: Calculate sum of all elements of temp and store it in variable loss \\
Step6: Calculate the gradient of 'w' w.r.t function loss

In [ ]:
from IPython.utils import tempdir
##DO NOT CHANGE SEED
torch.manual_seed(0)
labels = torch.randint(low=-800,high=50,size=(3,1)).type(torch.float)

## Your code starts here ##
w = torch.randn(3)
## Your code ends here ##

w = w.requires_grad_()

## Your code starts here ##
X = torch.rand(3, 3)

pred = torch.matmul(X, w)
print(pred.shape)

temp = torch.linalg.norm(labels-pred, dim=1, ord=1)

loss = temp.sum()
## Your code ends here ##

loss.backward(retain_graph=True)


## Your code starts here ##
grad = torch.autograd.grad(loss, w)
## Your code ends here ##
print(grad)

torch.Size([3])
(tensor([4.4869, 2.1056, 3.4464]),)


In [ ]:
from IPython.utils import tempdir
##DO NOT CHANGE SEED
torch.manual_seed(0)
labels = torch.randint(low=-800,high=50,size=(3,1)).type(torch.float)

## Your code starts here ##
w = torch.randn(3)
## Your code ends here ##

w = w.requires_grad_()

## Your code starts here ##
X = torch.rand(3, 3)

pred = torch.matmul(X, w)

temp = torch.abs(labels - pred)

loss = temp.sum()
## Your code ends here ##

loss.backward(retain_graph=True)

## Your code starts here ##
grad = torch.autograd.grad(loss, w)
## Your code ends here ##
print(grad)
print(pred.size())

(tensor([4.4869, 2.1056, 3.4464]),)
torch.Size([3])


### Q3. What is gradient of 'w' w.r.t loss function?
a)  [-4.4869, -2.1056, -3.4464] \\
b)  [4.4869, 2.1056, 3.4464] \\
c)  [-5.3670, -6.2075, -2.4481] \\
d)  [5.3670, 6.2075, 2.4481]

---

You will be training a 1-hidden layer multi-layer perceptron on a randomly generated dataset.

In [ ]:
# Please DO NOT modify this cell.

import os
import os.path as osp
import random

import numpy as np
import torch

def set_seed(seed: int):

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
# Please DO NOT modify this cell.

num_features = 10
classes = [0, 1, 2, 3, 4]
num_classes = len(classes)

num_samples = 100
num_train = 70
num_test = num_samples - num_train

In [ ]:
# Please DO NOT modify this cell.
# We are creating a random feature set and a random label set.
# The features and labels have no semantic meaning and might as well be garbage.

set_seed(2022)

features = np.random.random_sample((num_samples, num_features))
labels = np.random.choice(classes, size = num_samples)

# Train-test split
x_train = features[:num_train]
x_test = features[num_train:num_samples]

x_train = torch.Tensor(x_train)
x_test = torch.Tensor(x_test)

y_train = labels[:num_train]
y_test = labels[num_train:num_samples]

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
# Please DO NOT modify this cell.

print(f"Train features: {x_train.shape}")
print(f"Test features: {x_test.shape}")

print(f"Train labels: {y_train.shape}")
print(f"Train labels: {y_test.shape}")


Train features: torch.Size([70, 10])
Test features: torch.Size([30, 10])
Train labels: torch.Size([70])
Train labels: torch.Size([30])


Create train and test TensorDatasets from the respective numpy arrays

In [ ]:

#### YOUR CODE STARTS HERE ####
from torch.utils.data import TensorDataset

# Create train and test TensorDatasets
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

#### YOUR CODE ENDS HERE ####

Create dataloaders using the datasets created in the previous cell.
Use a batch size of 64

In [ ]:
#### YOUR CODE STARTS HERE ####

from torch.utils.data import DataLoader

batch_size = 64  # You can adjust this based on your preference and available memory

train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


#### YOUR CODE ENDS HERE ####

Network Definition

In [ ]:
# Please DO NOT modify this cell.

# Number of neurons in the hidden layer of our MLP
num_hidden = 512

Create the network MLP. Follow the comments to fill in the code

In [ ]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, num_features, num_classes, num_hidden):
        super(MLP, self).__init__()

        #### YOUR CODE STARTS HERE ####

        # define a linear layer with output channels as num_hidden
        self.hidden = nn.Linear(num_features, num_hidden)
        # Define a ReLU activation
        self.relu = nn.ReLU()
        # define a linear layer with output features corresponding to the number of classes
        self.classifier = nn.Linear(num_hidden, num_classes)

        #### YOUR CODE ENDS HERE ####

    def forward(self, x):
        # Use the layers defined above in a sequential way (folow the same as the layer definitions above) and
        # write the forward pass, use a relu activation after the hidden layer

        #### YOUR CODE STARTS HERE ####

        out = self.hidden(x)
        out = self.relu(out)
        out = self.classifier(out)

        #### YOUR CODE ENDS HERE ####

        return out

Training

In [ ]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    epoch_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):

        #### YOUR CODE STARTS HERE ####

        # send the data, target to the device
        data = data.to(device)
        target = target.to(device)

        # Flush out the gradients stored in optimizer
        optimizer.zero_grad()

        # Pass the batch data to the model and assign the output
        output = model(data)

        # Calculate the loss (use CrossEntropyLoss in PyTorch)
        loss = criterion(output, target)

        # Do a backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        #### YOUR CODE ENDS HERE ####

        # Store loss
        epoch_loss += loss.item() * data.shape[0]

    print(f"Train Average Loss: {epoch_loss/len(train_loader.dataset):.2f}")

 Inference

In [ ]:
def test(model, device, test_loader, criterion, mode):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            #### YOUR CODE STARTS HERE ####

            # Send data, target to the device
            data = data.to(device)
            target = target.to(device)

            # Pass the batch data to the model and assign the output
            output = model(data)
            #### YOUR CODE ENDS HERE ####

            test_loss += criterion(output, target).item() * data.shape[0]  # sum up batch loss
            pred = output.argmax(dim = 1, keepdim = True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_acc = 100. * correct / len(test_loader.dataset)

    print(f"{mode} Average loss: {test_loss:.2f}")
    print(f"{mode} Accuracy: {correct}/{len(test_loader.dataset)} ({test_acc:.2f}%)")

In [ ]:
set_seed(2022)

num_epochs = 300

#### YOUR CODE STARTS HERE ####

# Check availability of GPU and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize MLP model
num_features = 10
num_classes = 5
num_hidden = 512
model = MLP(num_features, num_classes, num_hidden).to(device)

# Define Adam Optimizer with a learning rate of 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Define CrossEntropyLoss as the criterion
criterion = nn.CrossEntropyLoss()


#### YOUR CODE ENDS HERE ####

for epoch in range(1, num_epochs+1):
    print(f"\nEpoch: {epoch}/{num_epochs}")

    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion, mode = "Test")


Epoch: 1/300
Train Average Loss: 1.64
Test Average loss: 1.58
Test Accuracy: 12/30 (40.00%)

Epoch: 2/300
Train Average Loss: 1.61
Test Average loss: 1.59
Test Accuracy: 10/30 (33.33%)

Epoch: 3/300
Train Average Loss: 1.60
Test Average loss: 1.60
Test Accuracy: 10/30 (33.33%)

Epoch: 4/300
Train Average Loss: 1.59
Test Average loss: 1.61
Test Accuracy: 10/30 (33.33%)

Epoch: 5/300
Train Average Loss: 1.59
Test Average loss: 1.61
Test Accuracy: 10/30 (33.33%)

Epoch: 6/300
Train Average Loss: 1.58
Test Average loss: 1.61
Test Accuracy: 10/30 (33.33%)

Epoch: 7/300
Train Average Loss: 1.58
Test Average loss: 1.61
Test Accuracy: 10/30 (33.33%)

Epoch: 8/300
Train Average Loss: 1.57
Test Average loss: 1.61
Test Accuracy: 9/30 (30.00%)

Epoch: 9/300
Train Average Loss: 1.56
Test Average loss: 1.62
Test Accuracy: 9/30 (30.00%)

Epoch: 10/300
Train Average Loss: 1.56
Test Average loss: 1.62
Test Accuracy: 9/30 (30.00%)

Epoch: 11/300
Train Average Loss: 1.55
Test Average loss: 1.62
Test Acc

In [ ]:
print(sum(p.numel() for p in model.parameters()))

8197



### Q4. What are total number of parameters in the model?

a) 8197 \\
b) 18521 \\
c) 8356 \\
d) 9105

In [ ]:
test(model, device, train_loader, criterion, mode = "Train")

Train Average loss: 0.27
Train Accuracy: 70/70 (100.00%)


### Q5. Report the final train accuracy (Choose the closest option).

a) 58% \\
b) 93% \\
c) 100% \\
d) 89%

In [ ]:
test(model, device, test_loader, criterion, mode = "Test")

Test Average loss: 2.91
Test Accuracy: 9/30 (30.00%)


### Q6. Report the final test accuracy (Choose the closest option).

a) 30% \\
b) 40% \\
c) 10% \\
d) 70%